In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *

## 4.
Question 4
Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?

Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

Округлите ответ до четырёх знаков после десятичной точки.

In [2]:
2*(1-stats.norm.cdf(9.57, loc=9.5, scale=0.4/np.sqrt(160)))

0.026856695507523787

In [3]:
z = (9.57-9.5)/(0.4/np.sqrt(160))
p_value = 2*(1-stats.norm.cdf(abs(z)))
print('p-value: %.4f' % p_value)

p-value: 0.0269


## 5
Как вы считаете, это отличие в среднем уровне кальция в крови практически значимо?
Да, раз критерий Стьюдента говорит, что отличие значимо

## 6
Имеются данные о стоимости и размерах 53940 бриллиантов:

Отделите 25% случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На обучающей выборке настройте две регрессионные модели:

* линейную регрессию с помощью LinearRegression без параметров
* случайный лес из 10 деревьев с помощью RandomForestRegressor с random_state=1.

Какая из моделей лучше предсказывает цену бриллиантов? Сделайте предсказания на тестовой выборке, посчитайте модули отклонений предсказаний от истинных цен. Проверьте гипотезу об одинаковом среднем качестве предсказаний, вычислите достигаемый уровень значимости. Отвергается ли гипотеза об одинаковом качестве моделей против двусторонней альтернативы на уровне значимости $\alpha=0.05$?

In [4]:
diamonds = pd.read_table('diamonds.txt')
diamonds.describe()
diamonds.head()
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   carat   53940 non-null  float64
 1   depth   53940 non-null  float64
 2   table   53940 non-null  float64
 3   price   53940 non-null  int64  
 4   x       53940 non-null  float64
 5   y       53940 non-null  float64
 6   z       53940 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 2.9 MB


In [5]:
X = diamonds[['carat', 'depth', 'table', 'x', 'y', 'z']]
y = diamonds['price']

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=1)

In [7]:
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print(mean_absolute_error(y_test, y_pred_lr))

890.3764004285619


In [8]:
print(np.abs(y_test - y_pred_lr).describe())

count    13485.000000
mean       890.376400
std       1161.068461
min          0.035915
25%        208.160385
50%        485.297677
75%       1098.149270
max      18239.846360
Name: price, dtype: float64


In [9]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=1).fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print(mean_absolute_error(y_test, y_pred_rf))

779.7117583558185


In [10]:
print(np.abs(y_test - y_pred_rf).describe())

count    13485.000000
mean       779.711758
std       1113.574813
min          0.096000
25%        119.340000
50%        316.490000
75%        962.280000
max      13484.020000
Name: price, dtype: float64


In [11]:
stats.ttest_rel(np.abs(y_test - y_pred_lr), np.abs(y_test - y_pred_rf))

Ttest_relResult(statistic=18.03725974451148, pvalue=6.936823477520982e-72)

In [12]:
from statsmodels.stats.weightstats import CompareMeans, DescrStatsW
print ("95%% confidence interval: [%f, %f]" % DescrStatsW(np.abs(y_test - y_pred_lr) - np.abs(y_test - y_pred_rf)).tconfint_mean())

95% confidence interval: [98.638526, 122.690758]


## 3.

В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате. 

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён. 

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [13]:
from statsmodels.stats.proportion import proportion_confint

In [45]:
def proportions_diff_confint_ind(pos_1, n1, pos_2, n2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(pos_1) / n1
    p2 = float(pos_2) / n2 
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    
    return (left_boundary, right_boundary)

In [46]:
def proportions_diff_z_stat_ind(pos_1, n1, pos_2, n2):
    p1 = float(pos_1) / n1
    p2 = float(pos_2) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [47]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [48]:
pos_1, n1 = 10, 34
pos_2, n2 = 4, 16
print(proportions_diff_z_stat_ind(pos_1, n1, pos_2, n2))
print("p-value: %.4f" % proportions_diff_z_test(proportions_diff_z_stat_ind(pos_1, n1, pos_2, n2), 'greater'))

0.32410186177608225
p-value: 0.3729


## 4
Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. На рисунке ниже показаны измеренные признаки.

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

* логистическая регрессия по признакам X_1, X_2, X_3;
* логистическая регрессия по признакам X_4, X_5, X_6.

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

In [49]:
banknotes = pd.read_table('banknotes.txt')
banknotes.describe()
banknotes.head()
banknotes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      200 non-null    float64
 1   X2      200 non-null    float64
 2   X3      200 non-null    float64
 3   X4      200 non-null    float64
 4   X5      200 non-null    float64
 5   X6      200 non-null    float64
 6   real    200 non-null    int64  
dtypes: float64(6), int64(1)
memory usage: 11.1 KB


In [50]:
X = banknotes[['X1', 'X2', 'X3', 'X4', 'X5', 'X6']]
y = banknotes['real']

In [51]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=50, random_state=1)

In [52]:
X_train_1 = X_train[['X1', 'X2', 'X3']]
X_train_2 = X_train[['X4', 'X5', 'X6']]
X_test_1 = X_test[['X1', 'X2', 'X3']]
X_test_2 = X_test[['X4', 'X5', 'X6']]

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [129]:
lr_1 = LogisticRegression().fit(X_train_1, y_train)
y_pred_1 = lr_1.predict(X_test_1)
print(1-accuracy_score(y_pred_1, y_test))
err_1 = np.array( [1 if y_train.values[i] == y_pred_1[i] else 0 for i in range(len(y_pred_1))] )
print(err_1, err_1.sum())

0.19999999999999996
[1 0 0 0 0 1 0 0 0 0 1 1 1 1 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1 1 1 0
 1 1 0 1 0 1 0 0 1 1 1 1 1] 24


In [130]:
lr_2 = LogisticRegression().fit(X_train_2, y_train)
y_pred_2 = lr_2.predict(X_test_2)
print(1-accuracy_score(y_pred_2, y_test))
err_2 = np.array( [1 if y_train.values[i] == y_pred_2[i] else 0 for i in range(len(y_pred_2))] )
print(err_2, err_2.sum())

0.020000000000000018
[1 0 0 0 0 1 0 0 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 0 0 1 1 0 0 0 0 0 0 1 1 1 0
 0 1 0 0 0 1 0 0 0 1 0 1 1] 21


In [135]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [136]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [137]:
print("95%% confidence interval for a difference between proportions: [%f, %f]" \
      % proportions_diff_confint_rel(err_1, err_2))

95% confidence interval for a difference between proportions: [-0.056416, 0.176416]


In [138]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(err_1, err_2)))

p-value: 0.312422


## 6.

Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов. 

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки. 

In [32]:
avg_mark = 541.4
avg_mark_data = np.ones(100) * avg_mark

In [33]:
exp_mark = 525.
st_dev = 100.

In [34]:
def z_test(mean_val, exp_val, st_dev, num):
    standard_error = st_dev / np.sqrt(num)
    return (mean_val - exp_val) / standard_error

In [35]:
def p_val_greater(z_stat):
    return 1 - stats.norm.cdf(z_stat)

In [36]:
z_stat = z_test(avg_mark, exp_mark, st_dev, (len(avg_mark_data)))
z_stat

1.6399999999999977

In [37]:
p_val = p_val_greater(z_stat)
p_val

0.05050258347410397

In [38]:
avg_mark = 541.5
avg_mark_data = np.ones(100) * avg_mark

In [39]:
z_stat = z_test(avg_mark, exp_mark, st_dev, (len(avg_mark_data)))
z_stat

1.65

In [40]:
p_val = p_val_greater(z_stat)
p_val

0.0494714680336481

## Часть 2
##  4
Давайте вернёмся к данным выживаемости пациентов с лейкоцитарной лимфомой из видео про критерий знаков:

49, 58, 75, 110, 112, 132, 151, 276, 281, 362^*
 
Измерено остаточное время жизни с момента начала наблюдения (в неделях); звёздочка обозначает цензурирование сверху — исследование длилось 7 лет, и остаточное время жизни одного пациента, который дожил до конца наблюдения, неизвестно.

Поскольку цензурировано только одно наблюдение, для проверки гипотезы H_0: med X = 200. 
На этих данных можно использовать критерий знаковых рангов — можно считать, что время дожития последнего пациента в точности равно 362, на ранг этого наблюдения это никак не повлияет. 

Критерием знаковых рангов проверьте эту гипотезу против двусторонней альтернативы, введите достигаемый уровень значимости, округлённый до четырёх знаков после десятичной точки.

Для классификатора используйте solver='liblinear'
Укажите для критерия знаковых рангов Вилкоксона mode='approx'

In [200]:
from scipy.stats import wilcoxon

In [201]:
l_w = np.array([49, 58, 75, 110, 112, 132, 151, 276, 281, 362])

In [202]:
print("M: %d, p-value: %f" % wilcoxon(l_w - 200, mode='approx'))

M: 17, p-value: 0.284503


## 5
В ходе исследования влияния лесозаготовки на биоразнообразие лесов острова Борнео собраны данные о количестве видов деревьев в 12 лесах, где вырубка не ведётся:

22, 22, 15, 13, 19, 19, 18, 20, 21, 13, 13, 15

и в 9 лесах, где идёт вырубка:

17, 18, 18, 15, 12, 4, 14, 15, 10.

Проверьте гипотезу о равенстве среднего количества видов в двух типах лесов против односторонней альтернативы о снижении биоразнообразия в вырубаемых лесах. Используйте ранговый критерий. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки.

In [203]:
forest_not_cut = np.array([22, 22, 15, 13, 19, 19, 18, 20, 21, 13, 13, 15])
forest_cut = np.array([17, 18, 18, 15, 12, 4, 14, 15, 10])

stats.mannwhitneyu(forest_not_cut, forest_cut, alternative='greater')

MannwhitneyuResult(statistic=81.0, pvalue=0.02900499272087373)

## 6
28 января 1986 года космический шаттл "Челленджер" взорвался при взлёте. Семь астронавтов, находившихся на борту, погибли. В ходе расследования причин катастрофы основной версией была неполадка с резиновыми уплотнительными кольцами в соединении с ракетными ускорителями. Для 23 предшествовавших катастрофе полётов "Челленджера" известны температура воздуха и появление повреждений хотя бы у одного из уплотнительных колец.

С помощью бутстрепа постройте 95% доверительный интервал для разности средних температур воздуха при запусках, когда уплотнительные кольца повреждались, и запусках, когда повреждений не было. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после запятой.

Чтобы получить в точности такой же доверительный интервал, как у нас:

установите random seed = 0 перед первым вызовом функции get_bootstrap_samples, один раз
сделайте по 1000 псевдовыборок из каждой выборки.

In [149]:
challenger = pd.read_csv('challenger.txt', delimiter='\t')

In [153]:
challenger_broken = np.array(challenger[challenger.Incident == 1]['Temperature'])
challenger_not_broken = np.array(challenger[challenger.Incident == 0]['Temperature'])

In [155]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [163]:
np.random.seed(0)
challenger_broken_bs_mean = np.array(list(map(np.mean, get_bootstrap_samples(challenger_broken, 1000))))
challenger_not_broken_bs_mean = np.array(list(map(np.mean, get_bootstrap_samples(challenger_not_broken, 1000))))

In [170]:
print('95%% confidence interval for times decrease of infarction: %s' %
      stat_intervals(challenger_broken_bs_mean - challenger_not_broken_bs_mean, 0.05))

95% confidence interval for times decrease of infarction: [-8.06457589 -1.45040179]


## 7

На данных предыдущей задачи проверьте гипотезу об одинаковой средней температуре воздуха в дни, когда уплотнительный кольца повреждались, и дни, когда повреждений не было. Используйте перестановочный критерий и двустороннюю альтернативу. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки. 

Чтобы получить такое же значение, как мы:

установите random seed = 0;
возьмите 10000 перестановок.

In [178]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [179]:
def get_random_combinations(n1, n2, max_combinations):
    index = np.arange(n1 + n2)
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [180]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [181]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [183]:
np.random.seed(0)
print('p-value: %.4f' % permutation_test(challenger_broken, challenger_not_broken, max_permutations=10000))

p-value: 0.0057
